# `Market-Report-Dev.ipynb`

Market report 関連開発用 notebook


In [29]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import sys, os, re, json, yaml, datetime, warnings
import curl_cffi
import pandas as pd
pd.options.display.precision = 2
import sqlite3
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from fredapi import Fred
from edgar import *
from edgar.reference import CompanySubset
set_identity('youxitiankaggle@gmail.com')
import yfinance as yf
from ddgs import DDGS
warnings.simplefilter('ignore')
from dotenv import load_dotenv
load_dotenv()


ROOT_DIR = Path(os.environ.get("ROOT_DIR")) # type: ignore
DATA_DIR = Path(os.environ.get("DATA_DIR")) # type: ignore
FRED_DIR = Path(os.environ.get("FRED_DIR")) # type: ignore
BLOOMBERG_DIR = Path(os.environ.get("BLOOMBERG_DIR")) # type: ignore
TSA_DIR = Path(os.environ.get("TSA_DIR")) # type: ignore

sys.path.append(str(ROOT_DIR))

from src import market_report_utils as mru
from src import fred_database_utils as fred_utils
from src import us_treasury



fred_db_path = FRED_DIR / "FRED.db"
tsa_db_path = TSA_DIR / "TSA.db"
sp_price_db_path = BLOOMBERG_DIR / "SP_Indices_Price.db"

# daily data collect
fred = fred_utils.FredDataProcessor()
# fred.store_fred_database()

bbg = mru.BloombergDataProcessor()
# bbg.delete_table_from_db(db_path=bbg.sp_price_db, table_name="Members")
bbg.store_sp_indices_price_to_database()
sp500_tickers = [re.split(" ", s)[0] for s in bbg.get_current_sp_members()]
# display(sp500_tickers)

tsa_data_collector = mru.TSAPassengerDataCollector()
df = tsa_data_collector.scrape_tsa_passenger_data()
tsa_data_collector.store_to_tsa_database(df=df, db_path=tsa_db_path)

analyzer = mru.MarketPerformanceAnalyzer()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
An error occurred: 'Date'
Reading data from 'SP500-Indices.xlsx'...
Successfully loaded and reshaped 850 rows from Excel.
Database latest date: 2025-11-03. Found 0 new rows to add.
-------------------- + TSA Passenger Data + --------------------
追加する新規データはありませんでした。


[*********************100%***********************]  34 of 34 completed


## Edgartools

SEC からティッカー一覧を取得し yfinance でヒストリカルデータを取得する


In [ ]:
# 複数条件の組み合わせ
companies = (
    CompanySubset().from_exchange("NYSE")  # NYSE上場
    # .from_industry(sic=2834)  # 製薬業界
    # .filter_by(lambda df: df[df["state_of_incorporation"] == "DE"])  # デラウェア州法人
    # .sample(10)  # 10社をサンプリング
    .get()
)
display(companies)

analyzer = mru.MarketPerformanceAnalyzer()
df = analyzer.yf_download_with_curl(
    tickers_to_download=companies["ticker"].unique().tolist()
)
display(df)


,cik,ticker,name,exchange
0,1067983,BRK-B,BERKSHIRE HATHAWAY INC,NYSE
1,59478,LLY,ELI LILLY & Co,NYSE
2,19617,JPM,JPMORGAN CHASE & CO,NYSE
3,104169,WMT,Walmart Inc.,NYSE
4,1403161,V,VISA INC.,NYSE
...,...,...,...,...
3253,2074697,FCRS-WT,FutureCrest Acquisition Corp.,NYSE
3254,2074872,KRSP-UN,Rice Acquisition Corp 3,NYSE
3255,2075336,YCY-UN,AA Mission Acquisition Corp. II,NYSE
3256,2075336,YCY-WT,AA Mission Acquisition Corp. II,NYSE


[*********************100%***********************]  34 of 34 completed
[*********************100%***********************]  3257 of 3258 completed

188 Failed downloads:
['BIO-B', 'CVX', 'KAR', 'CIEN', 'MOV', 'SPME', 'EBS', 'IVR', 'IIIN', 'GPC', 'YELP', 'MOGU', 'SYF-PA', 'COPL', 'JNJ', 'ATMU', 'ASIC', 'MSCI', 'BRC', 'WOR', 'CRF', 'CNR', 'GBAB', 'JEF', 'AMN', 'PSBD', 'ANVS', 'CICB', 'BRSL', 'TFII', 'MA', 'EFC-PC', 'SLAI', 'BBBY', 'PAYC', 'PK', 'TAGS', 'GS-PA', 'SARO', 'VNO-PO', 'MLR', 'MRK', 'SMFG', 'WWR', 'KEY-PI', 'XPEV', 'SOLV', 'TALO', 'DNA', 'IIPR', 'SKYH', 'KEY-PL', 'NMAX', 'WTS', 'SON', 'PFN', 'EDN', 'FIG', 'WU', 'VYLD', 'PCN', 'RITM-PB', 'TRAK', 'DAVA', 'AQN', 'WBS', 'FISK', 'BWA', 'EBR-B', 'UZF', 'EVTL', 'AGM-PH', 'CSR', 'PNI', 'OPY', 'DXC', 'MUFG', 'PBI', 'CVU', 'NRUC', 'UNL', 'WY', 'MT', 'CRI', 'BNY', 'CRD-B', 'EHAB', 'DAL', 'KKR-PD', 'MUE', 'FXY', 'STM', 'VVV', 'RC-PE', 'NKE', 'BW-PA', 'TDS-PV', 'BMO', 'BML-PH', 'BLX', 'CPNG', 'ORC', 'NBR', 'GEL', 'PII', 'NUVB-WT', 'ODC', 'MS

,Date,Ticker,variable,value
0,2023-11-22,A,Adj Close,1.22e+02
1,2023-11-22,AA,Adj Close,2.57e+01
2,2023-11-22,AAM,Adj Close,NaN
3,2023-11-22,AAM-UN,Adj Close,NaN
4,2023-11-22,AAM-WT,Adj Close,NaN
...,...,...,...,...
9813091,2025-11-21,ZTO,Volume,1.71e+06
9813092,2025-11-21,ZTR,Volume,2.18e+05
9813093,2025-11-21,ZTS,Volume,6.37e+06
9813094,2025-11-21,ZVIA,Volume,7.63e+05


## DuckDuckGo のテスト


In [ ]:
def search_news(keywords, time_limit="w"):
    results = []
    try:
        # keywords: 検索語句
        # region="wt-wt": 全世界 (日本限定なら "jp-jp")
        # safesearch="off": 制限なし
        # timelimit="d": 過去1日 (w=1週間, m=1ヶ月) → 指定しない場合は削除
        news_gen = DDGS().news(
            keywords=keywords,
            region="wt-wt",
            safesearch="off",
            timelimit=time_limit,
            backend="auto",
        )

        for r in news_gen:
            results.append(
                {
                    "title": r.get("title"),
                    "source": r.get("source"),
                    "date": r.get("date"),  # 記事の日付（重要）
                    "url": r.get("url"),
                    "body": r.get("body"),  # 記事の要約
                }
            )

    except Exception as e:
        return {"error": str(e)}

    return results


# if __name__ == "__main__":
#     # コマンドライン引数を結合して検索クエリにする
#     query = " ".join(sys.argv[1:])

#     if not query:
#         print(
#             json.dumps(
#                 {"error": "No query provided. Usage: python search_ddg.py [KEYWORDS]"}
#             )
#         )
#     else:
#         # 結果をJSON形式で標準出力する（Claudeが読み取る用）
#         data = search_news(query)
#         print(json.dumps(data, indent=2, ensure_ascii=False))


In [ ]:
def search_news(
    query: str, news_site_domain: str, time_limit: str = "m", max_results: int = 10
):

    results = DDGS().news(
        query=f"{query} site:{news_site_domain}",
        region="us-en",
        safesearch="off",
        timelimit=time_limit,
        page=1,
        backend="google",
        max_results=max_results,
    )

    return results


# results = search_news("Tesla", "https://www.bloomberg.com", time_limit="m")
results = search_news("Nvidia GPU", "https://www.reuters.com/", max_results=20)
print(len(results))
for res in results:
    print(f"\n\n{"="*20} {res['date']}, {res['title']}, {res['source']} {"="*20}\n")

    print(res["body"])
    print(f"URL: {res['url']}")


20


==================== 2025-11-19T12:03:00+00:00, Nvidia, Menlo Micro collaboration speeds up AI chip testing, Reuters ====================

Nvidia and Menlo Micro on Wednesday said they have used technology from the startup to dramatically speed up the testing of AI chips, easing a significant production bottleneck.
URL: https://www.reuters.com/business/nvidia-menlo-micro-collaboration-speeds-up-ai-chip-testing-2025-11-19/


==================== 2025-11-17T10:05:00+00:00, GMI Cloud to build $500 million AI data centre in Taiwan with Nvidia chips, Reuters ====================

U.S.-based cloud services provider GMI Cloud said on Monday it will build a $500 million artificial intelligence data centre in Taiwan with the support of U.S. chipmaker Nvidia .
URL: https://www.reuters.com/world/asia-pacific/gmi-cloud-build-500-million-ai-data-centre-in-taiwan-with-nvidia-chips-2025-11-17/


==================== 2025-11-20T23:30:00+00:00, Nvidia beat may yet stir fear on the Street, Reuters 

In [ ]:
print(len(results))
for res in results[:2]:
    print(f"\n\n{"="*20} {res['date']}, {res['title']}, {res['source']} {"="*20}\n")

    print(res["body"])
    print(f"URL: {res['url']}")

    response = requests.get(res["url"])
    soup = BeautifulSoup(response.content, "html.parser")
    display(str(soup))


20


==================== 2025-11-19T12:03:00+00:00, Nvidia, Menlo Micro collaboration speeds up AI chip testing, Reuters ====================

Nvidia and Menlo Micro on Wednesday said they have used technology from the startup to dramatically speed up the testing of AI chips, easing a significant production bottleneck.
URL: https://www.reuters.com/business/nvidia-menlo-micro-collaboration-speeds-up-ai-chip-testing-2025-11-19/


'<html lang="en"><head><title>reuters.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAPloVLo7w0OQA-jUpmA==\',\'hsh\':\'2013457ADA70C67D6A4123E0A76873\',\'t\':\'bv\',\'qp\':\'\',\'s\':46743,\'e\':\'1192797090ea69436aa5edfcdac0f58fc493bac058031c8df339704a1125886f55f9de94c31c8e3aacb8be1dacb2e3fb\',\'host\':\'geo.captcha-delivery.com\',\'cookie\':\'wKtfmeS5qkMk~hORyoT2hzE6GRS6zqUjGwxHyToZyJDlybKVZ7pfZjUjAzL1Lp_KTP0imnAaBJVHvjk1dmQKVjNuu4~858Jo2vnuwj1VBboHb7A1yEgc81t3sCg85EQs\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>'



==================== 2025-11-17T10:05:00+00:00, GMI Cloud to build $500 million AI data centre in Taiwan with Nvidia chips, Reuters ====================

U.S.-based cloud services provider GMI Cloud said on Monday it will build a $500 million artificial intelligence data centre in Taiwan with the support of U.S. chipmaker Nvidia .
URL: https://www.reuters.com/world/asia-pacific/gmi-cloud-build-500-million-ai-data-centre-in-taiwan-with-nvidia-chips-2025-11-17/


'<html lang="en"><head><title>reuters.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAduTed4hIww0A-jUpmA==\',\'hsh\':\'2013457ADA70C67D6A4123E0A76873\',\'t\':\'bv\',\'qp\':\'\',\'s\':46743,\'e\':\'38547069b8610df8abb8a3fe515731ce372be822b39b9470ca42b6c9c93f8937c1378ade05f0a571e8f213576efea188\',\'host\':\'geo.captcha-delivery.com\',\'cookie\':\'BN8ekIRPATtfCY20aQt4Yu~~V2UK271hEEJXtaxU_tGJv9FE~Z_1hXviYGYL3uhGYxUp8ywIwpU0eCZu~zkMpNwm0QTsg_U8J2CAyKkY3pzFTm~xEcVTZUR0vPhjjIho\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>'

## Metal Commodities

-   correlation の FFT


In [ ]:
df_corr_copy = df_corr.copy()
ticker = "GLD"
for ticker in ["GLD"]:
    df_slice = df_corr_copy.loc[df_corr_copy["Ticker"] == ticker]
    df_slice = pd.pivot(df_slice, index="Date", columns="variable", values="value")

    for variable in df_slice.columns:
        print(variable)
        fft_result = calculate_period_from_fft(df_slice, column_name=variable)


In [ ]:
def calculate_period_from_fft(df: pd.DataFrame, column_name: str):
    """
    データフレームの指定された列に対してFFTを適用し、最も強い周期成分を計算します。

    Args:
        df (pd.DataFrame): 時系列データを含むデータフレーム。
        column_name (str): FFTを適用する列の名前（相関係数）。

    Returns:
        float: 最も強い周波数成分に対応する周期（日）。
    """
    # 1. データ準備
    # 相関係数の時系列データを取得
    signal = df[column_name].values
    N = len(signal)  # サンプル数 (データ点数)

    # 日次データなので、サンプリング間隔 T は 1日
    # T はサンプリング周波数 (fs) の逆数: fs=1/T
    T = 1.0  # サンプリング間隔（日）

    # 2. 離散フーリエ変換 (DFT) の実行
    # FFTの結果は複素数配列
    fft_result = np.fft.fft(signal)

    # 3. 周波数スケールの生成
    # FFTの結果に対応する周波数の配列を生成
    # freqs は 1/T の単位、つまり「1/日」
    freqs = np.fft.fftfreq(N, d=T)

    # 4. パワースペクトルの計算とピークの探索
    # パワースペクトルは |FFTの結果|^2 で計算されるが、一般的に絶対値 (振幅) を使うことが多い。
    # ここでは振幅の絶対値の2乗（パワー）を使用。
    power_spectrum = np.abs(fft_result) ** 2

    # 負の周波数成分は対称的なので、正の周波数成分のみを考慮する
    # ゼロ周波数（直流成分、平均値に対応）を除外
    # freqs[1:N//2] は正の周波数成分
    # power_spectrum[1:N//2] はそれに対応するパワー
    positive_freqs = freqs[1 : N // 2]
    positive_power = power_spectrum[1 : N // 2]

    # 最もパワーが大きい周波数成分のインデックスを見つける
    # このインデックスは positive_power, positive_freqs の中のインデックス
    idx_max_power = np.argmax(positive_power)

    # 最も強い周波数成分を取得
    dominant_frequency = positive_freqs[idx_max_power]

    # 5. 周期の計算
    # 周期 P (日) = 1 / 周波数 f (1/日)
    if dominant_frequency > 0:
        dominant_period = 1 / dominant_frequency
    else:
        # 周波数がゼロの場合は、非常に長い周期（トレンド）
        dominant_period = np.inf

    print(f"✅ 最も強い周波数: {dominant_frequency:.4f} [1/日]")
    print(f"✅ 対応する周期: {dominant_period:.2f} [日]")

    # (オプション) パワースペクトルと周波数の可視化
    plt.figure(figsize=(10, 4))
    plt.plot(positive_freqs, positive_power)
    plt.title("Power Spectrum (Positive Frequencies)")
    plt.xlabel("Frequency (1/day)")
    plt.ylabel("Power")
    plt.grid(True)
    plt.show()

    return dominant_period


## 分足


In [ ]:
price_min = analyzer.yf_download_with_curl(interval="1h", period="max")

ticker = "^SPX"
df_plot = price_min[ticker]
fig = go.Figure(
    data=[
        go.Candlestick(
            x=df_plot.index,
            open=df_plot["Open"],
            high=df_plot["High"],
            low=df_plot["Low"],
            close=df_plot["Close"],
            name=ticker,
        )
    ]
)
fig.update_layout(
    width=1200,
    height=600,
    template="plotly_dark",
    xaxis_type="category",
    yaxis_title="Price(USD)",
)


## DataFrame 画像保存テスト


In [ ]:
def save_df_as_image(
    dataframe,
    filename,
    figsize=(10, 6),
    font_size=12,
    header_color="#40466E",
    row_colors=["#f1f1f2", "#ffffff"],
    edge_color="#e0e0e0",
    header_height=0.08,
    row_height=0.06,
):
    """
    PandasデータフレームをPNG画像として保存します。

    Args:
        dataframe (pd.DataFrame): 保存するデータフレーム。
        filename (str): 出力ファイル名 (例: 'output.png')。
        figsize (tuple): 画像のサイズ (幅, 高さ)。
        font_size (int): フォントサイズ。
        header_color (str): ヘッダーの背景色。
        row_colors (list): 行の背景色 (ストライプ表示用)。
        edge_color (str): セルの境界線の色。
        header_height (float): ヘッダーの高さの割合。
        row_height (float): 各行の高さの割合。
    """

    # 画像サイズをデータフレームの行数と列数に応じて調整する
    # デフォルトのサイズよりはみ出す場合を考慮
    num_rows = len(dataframe)
    num_cols = len(dataframe.columns)

    # 行数や列数に応じてfigsizeを自動調整する基本的なロジック
    # 細かい調整は手動で行う方が良い場合もあります
    adjusted_width = max(figsize[0], num_cols * 1.5)  # 1列あたり1.5インチ
    adjusted_height = max(figsize[1], num_rows * row_height * 10)  # 行数に応じて調整

    fig, ax = plt.subplots(figsize=(adjusted_width, adjusted_height))
    ax.axis("off")  # 軸を非表示にする

    # テーブルの描画
    table = ax.table(
        cellText=dataframe.values,
        colLabels=dataframe.columns,
        loc="center",
        cellLoc="left",  # セルのテキストを左寄せ
        rowLoc="left",  # 行のテキストを左寄せ
    )

    # テーブルの書式設定
    table.auto_set_font_size(False)
    table.set_fontsize(font_size)
    table.scale(1, 1.5)  # セルの幅と高さを調整 (幅, 高さ)

    # セルのスタイル設定
    for i, key in enumerate(table.get_celld().keys()):
        cell = table.get_celld()[key]
        cell.set_edgecolor(edge_color)
        cell.set_linewidth(0.5)  # 境界線の太さ

        if key[0] == 0:  # ヘッダー行
            cell.set_facecolor(header_color)
            cell.set_text_props(
                weight="bold", color="white"
            )  # ヘッダーのテキストを太字白に
            cell.set_height(header_height)
        else:  # データ行
            cell.set_facecolor(row_colors[key[0] % len(row_colors)])  # ストライプ
            cell.set_height(row_height)

    plt.savefig(
        filename, bbox_inches="tight", pad_inches=0.1, dpi=300
    )  # dpiで解像度指定
    plt.close(fig)  # グラフを閉じ、メモリを解放

    print(f"データフレームを '{filename}' として保存しました。")


# --- 使用例 ---
save_df_as_image(
    performance_dict["SP500_Indices"],
    ROOT_DIR / "dataframe_output.png",
    figsize=(8, len(performance_dict["SP500_Indices"]) * 0.5 + 1),
    font_size=10,
)

# より多くの行がある場合の例
long_df = pd.DataFrame(np.random.rand(20, 5), columns=[f"Col{i}" for i in range(5)])
save_df_as_image(long_df, "long_dataframe_output.png", font_size=8)
